<a href="https://colab.research.google.com/github/JJmikey/todoAPI/blob/main/APITest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask
!pip install flask-ngrok

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive


In [3]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2024-01-23 04:41:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  4.86MB/s    in 2.7s    

2024-01-23 04:41:38 (4.86 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip


In [4]:
!/ngrok authtoken 2bJNpHf4hTPhIqK2OEt5azgduaH_3uUQZhYfhSRNWtC7UVRoA


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!pip install flask_ngrok

In [13]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/<name>")
def home(name):
    return f"<h1>hello {name}</h1>"

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b7f0-34-125-11-169.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:27:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:27:12] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:27:23] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:27:23] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:28:55] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:28:56] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:30:25] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:30:25] "GET /favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:32:06] "GET /bing HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 03:32:06] "GET /favicon.ico HTTP/1.1" 200 -


In [7]:
from flask import Flask, jsonify

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return jsonify(message="Hello, World!")
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://180b-35-240-144-59.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 04:43:11] "GET / HTTP/1.1" 200 -


In [27]:
from flask import Flask, jsonify, request
#from flask_ngrok import run_with_ngrok

app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run

todo_tasks = []
current_task_id = 0 # A global variable to store the current task id

@app.route("/tasks", methods=['GET', 'POST'])
def manage_tasks():
    global current_task_id # Use the global variable
    if request.method == 'GET':
        return jsonify(todo_tasks)
    elif request.method == 'POST':
        task = request.json.get('task', '')
        if task:
            current_task_id += 1 # Increment the current task id
            todo_tasks.append({'id': current_task_id, 'task': task, 'status': 'pending'}) # Add the task with the id
            return jsonify({'message': 'Task added', 'id': current_task_id}), 201 # Return the id of the added task
        else:
            return jsonify({'message': 'Task is required'}), 400


@app.route("/tasks/<int:id>", methods=['PUT', 'DELETE'])
def modify_task(id):
    task = [t for t in todo_tasks if t['id'] == id] # Find the task by id
    if len(task) == 0: # If no task is found, return an error message and status code
        return jsonify({'message': 'Task not found'}), 404
    if request.method == 'PUT': # If the request method is PUT, update the task
        task[0]['task'] = request.json.get('task', task[0]['task']) # Update the task content
        task[0]['status'] = request.json.get('status', task[0]['status']) # Update the task status
        return jsonify({'message': 'Task updated'}), 200
    elif request.method == 'DELETE': # If the request method is DELETE, remove the task
        todo_tasks.remove(task[0]) # Remove the task from the list
        return jsonify({'message': 'Task deleted'}), 200

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=int(os.environ.get('PORT', 8080))) #for deploy on vercel

#if __name__ == "__main__":
#       app.run()   (run with grok先需要)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9cf4-35-240-144-59.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:10:54] "GET /tasks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:06] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:18] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:25] "GET /tasks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:32] "POST /tasks HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:39] "GET /tasks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:11:59] "POST /tasks HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:12:01] "GET /tasks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:12:56] "GET /tasks/1 HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:17:06] "POST /tasks/1 HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:17:32] "PUT /tasks/1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jan/2024 06:17

In [29]:
!mkdir my_project
!cp -r /content/* my_project/

mkdir: cannot create directory ‘my_project’: File exists
^C


# 新增區段